In [21]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
from sklearn.metrics import mean_squared_error
import mlflow
import mlflow.xgboost
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

In [22]:
data = pd.read_csv(r'flats_to_buy_preproceesed.csv')


data.dropna(subset=['ConstructionYear'], inplace=True)
data.dropna(subset=['Object_price'], inplace=True)
data.dropna(subset=['Rooms'], inplace=True)
data.drop('Title', axis=1, inplace=True)
data.drop('Object_currency', axis=1, inplace=True)
data.drop('Url', axis=1, inplace=True)
data.drop('EstateType', axis=1, inplace=True)
data.drop('DistributionType', axis=1, inplace=True)
data = data.reindex()
data = data.reset_index(drop=True)

y = data['Object_price']
X = data.drop('Object_price', axis=1)

In [23]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                  train_size=0.8, 
                                                  random_state = 0)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  train_size=0.8, 
                                                  random_state = 0)

num_cols = [col for col in train_X.columns if train_X[col].dtype == 'float64']
cat_cols = [col for col in train_X.columns if train_X[col].dtype == 'object']
bin_cols = [col for col in train_X.columns if train_X[col].dtype == 'int64']

In [24]:
encoder = OneHotEncoder(handle_unknown='ignore')
X_train_categorical_xgb = encoder.fit_transform(train_X[cat_cols]).toarray()
X_valid_categorical_xgb = encoder.transform(val_X[cat_cols]).toarray()
X_test_categorical_xgb = encoder.transform(test_X[cat_cols]).toarray()
X_train_processed = np.concatenate([train_X[num_cols], X_train_categorical_xgb], axis=1)
X_valid_processed = np.concatenate([val_X[num_cols], X_valid_categorical_xgb], axis=1)
X_test_processed = np.concatenate([test_X[num_cols], X_test_categorical_xgb], axis=1)
X_train_processed = np.concatenate([train_X[bin_cols], X_train_processed], axis=1)
X_valid_processed = np.concatenate([val_X[bin_cols], X_valid_processed], axis=1)
X_test_processed = np.concatenate([test_X[bin_cols], X_test_processed], axis=1)

In [25]:
mlflow.set_tracking_uri("http://localhost:5000")

In [26]:
mlflow.end_run()


# XGBoost

In [27]:
mlflow.xgboost.autolog()
mlflow.start_run()
model = xgb.XGBRegressor(eval_metric=['rmse', 'mae', 'logloss'], early_stopping_rounds=39)
model.fit(X=X_train_processed, 
          y=train_y,
          eval_set=[(X_valid_processed, val_y)],

          verbose=True)

mlflow.xgboost.log_model(model, "xgboost-model")

preds = model.predict(X_test_processed)
score = mean_absolute_error(test_y, preds)
mlflow.end_run()

2023/07/05 18:22:03 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.


[0]	validation_0-rmse:377643.68046	validation_0-mae:262610.76982	validation_0-logloss:-16881964.40244
[1]	validation_0-rmse:315303.28482	validation_0-mae:198007.32736	validation_0-logloss:-17089572.53659
[2]	validation_0-rmse:273814.66459	validation_0-mae:156059.12976	validation_0-logloss:-17182301.21951
[3]	validation_0-rmse:247366.68043	validation_0-mae:130843.66825	validation_0-logloss:-17234962.23171
[4]	validation_0-rmse:227769.44017	validation_0-mae:111355.84699	validation_0-logloss:-17270074.64634
[5]	validation_0-rmse:218264.07826	validation_0-mae:101177.79649	validation_0-logloss:-17289771.14634
[6]	validation_0-rmse:213008.86382	validation_0-mae:94187.91902	validation_0-logloss:-17302643.80488
[7]	validation_0-rmse:208510.08491	validation_0-mae:88551.81269	validation_0-logloss:-17312736.74390
[8]	validation_0-rmse:205527.83342	validation_0-mae:85034.76410	validation_0-logloss:-17320453.50000
[9]	validation_0-rmse:199592.69830	validation_0-mae:82180.19207	validation_0-logloss:

## Linear Regression


In [29]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Enable automatic logging
mlflow.sklearn.autolog()

# Start an MLflow run
mlflow.start_run()

# Train the Linear Regression model
model = LinearRegression()
model.fit(X_train_processed, train_y)

# Log the model
mlflow.sklearn.log_model(model, "linear-regression-model")

# Make predictions on the test set
preds = model.predict(X_test_processed)
score = mean_absolute_error(test_y, preds)

mlflow.end_run()

2023/07/05 18:26:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'Series' object has no attribute 'flatten'
